In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import Counter

import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.corpus import stopwords

from random import randint
import string
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
from jupyterthemes import jtplot



#read in one dataset
df=pd.read_csv('epicurious')

df=df[['categories','directions','ingredients','title']]

df=df.reset_index(drop=True)

df=df.dropna()

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import Counter

import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.corpus import stopwords

from random import randint
import string
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec

#read in one dataset
df=pd.read_csv('epicurious')

df=df[['categories','directions','ingredients','title']]

df=df.reset_index(drop=True)

df=df.dropna()

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
# get stop words


stopwords_list = stopwords.words('english')

#add punctuation to stopwords list
stopwords_list += list(string.punctuation)

#add words that aren't in the NLTK stopwords list
stopwords_list += ['unsalted','grated','minced','ground','thinly','ounces','finely','black','red','white','plus','whole','coursely','cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
stopwords_list+=['``',"'s","''",'drained','baking','coarsely','stick','green','packed','all-purpose','room','temperature','1/2-inch','''''','pounds','dry','divided','halved','trimmed','slices','1/2-inch-thick','1/4-inch-thick','s','12','1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','1/8','small','large','medium']

def process_article(article):
    tokens = word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

df['processed_titles']=df['title'].apply(lambda x: process_article(x))

# this is the best format so far
df['processed_ingredients']=df['ingredients'].apply(lambda x: process_article(x))

df['processed_directions']=df['directions'].apply(lambda x: process_article(x))

df['recipe_combo']=df['processed_titles']+df['processed_ingredients']+df['processed_directions']



model_combo = Word2Vec(df['recipe_combo'], min_count=1)

In [5]:
test_run=df[df['ingredients'].str.contains('bread')]

test_run=test_run.reset_index(drop=True)


item=randint(0,(len(test_run)-1))

test_ingredients=test_run['processed_ingredients'][item]

key=test_ingredients[3]

answer=model_combo.wv.most_similar(positive=[test_ingredients[0]])

final=answer[0][0]

similar_words=model_combo.wv.most_similar(positive=[key])

similar_list=[]
words=[]
for word in range(1,len(similar_words)):
    similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
    words.append(similar_words[word][0])

layered_relationships=dict(zip(words,similar_list))

final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))

test_run.iloc[item]['title'],test_run.iloc[item]['ingredients']

test_run.iloc[item]['title'].replace(key,final_2), test_run.iloc[item]['directions'].replace(key,final_2), test_run.iloc[item]['ingredients'].replace(key,final_2)

('Brussels Sprouts with Panko ',
 "['Pour 1/4 to 1/2 inch corn oil in a large skillet and place over a high heat until hot. To test the temperature, slip half a Brussels sprout into the pan; if it makes a popping sound, the oil is hot enough. Working in batches, fry the Brussels sprouts, turning occasionally, until they are browned all over, 2 to 3 minutes. Using a slotted spoon, transfer the sprouts to a paper towel-lined plate to drain.', 'Meanwhile, whisk together the Thick Tahini Sauce , yogurt and pomegranate molasses in a medium bowl. Set aside.', 'In a small skillet, heat the olive oil over medium-high until hot. Add the garlic and saute until fragrant, about 1 minute. Add the panko and stir constantly until the crumbs are golden brown, about 2 minutes. Stir in the salt and remove the breadcrumbs from the heat. Transfer to a paper towel-lined plate to cool.', 'Place the Brussels sprouts in a serving dish, drizzle with the sauce and top with the panko crumbs. Serve immediately.']

In [6]:
df['recipe_vector']=df['recipe_combo'].apply(lambda x: sum(model_combo.wv[x]))

recipe=df[['title','recipe_vector','ingredients','processed_ingredients','directions']]
recipe=recipe.reset_index(drop=True)
test_cluster=[]
for num in range(len(recipe['recipe_vector'])):
    test_cluster.append(recipe['recipe_vector'][num])

In [7]:
def recipe_generator(ingred):
    
    test_run=df[df['ingredients'].str.contains(ingred)]
    test_run=test_run.reset_index(drop=True)


    item=randint(0,(len(test_run)-1))

    test_ingredients=test_run['processed_ingredients'][item]

    key=test_ingredients[3]

    answer=model_combo.wv.most_similar(positive=[test_ingredients[0]])

    final=answer[0][0]

    similar_words=model_combo.wv.most_similar(positive=[key])

    similar_list=[]
    words=[]
    for word in range(1,len(similar_words)):
        similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
        words.append(similar_words[word][0])

    layered_relationships=dict(zip(words,similar_list))

    final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))

    test_run.iloc[item]['title'],test_run.iloc[item]['ingredients']

    unique=test_run.iloc[item]['title'].replace(key,final_2), test_run.iloc[item]['directions'].replace(key,final_2), test_run.iloc[item]['ingredients'].replace(key,final_2)
    return unique

In [76]:
from IPython.display import Javascript, display
from ipywidgets import widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Run Cells")
button.on_click(run_all)
display(button)

Button(description='Run Cells', style=ButtonStyle())

In [8]:
recipe_generator('chicken')

('Chestnut, Bacon, Dried Apple, and Corn Bread Stuffing ',
 "['Cook bacon in heavy large skillet over medium-high heat until brown and crisp. Using slotted spoon, transfer bacon to large bowl. Pour off all but 2 tablespoons drippings from skillet. Add onions and celery to drippings in skillet. Cover and cook over medium heat until vegetables are soft, stirring occasionally, about 12 minutes. Add vegetables to bacon; mix in apples, thyme, and sage, then chestnuts. (Can be made 1 day ahead. Cover and chill. Reheat to lukewarm before continuing.)', 'Add corn bread stuffing mix to chestnut mixture. Mix in 1 3/4 cups broth. Drizzle with butter.', 'Loosely fill main cavity and neck cavity of turkey with stuffing. Add enough broth to remaining stuffing to moisten slightly (1/4 cup to 3/4 cup, depending on amount of remaining stuffing). Generously butter baking dish. Spoon remaining stuffing into prepared dish. Cover dish with buttered foil, buttered side down. Bake stuffing in dish — alongsid

In [9]:
X=np.asarray(test_cluster)

def recipe_clusterer(num):
    k_means = KMeans(n_clusters = num)
    k_means.fit(X)
    y_hat = k_means.predict(X)
    cluster_numbers=[]
    for number in range(len(y_hat)):
        cluster_numbers.append(y_hat[number])
    recipe['clustered_groups']=pd.DataFrame(cluster_numbers)
    return recipe

In [52]:
def run_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

click = widgets.Button(description="Create next input")
click.on_click(run_next)
display(click)

Button(description='Create next input', style=ButtonStyle())

In [96]:

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Create next input")
button.on_click(run_all)
display(button)


Button(description='Create next input', style=ButtonStyle())

In [11]:
recipe_clusterer(100)

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
0,"Lentil, Apple, and Turkey Wrap","[10.785597, 21.088675, -40.898643, 66.33199, 1...",['4 cups low-sodium vegetable or chicken stock...,"[low-sodium, vegetable, chicken, stock, dried,...","['1. Place the stock, lentils, celery, carrot,...",23
1,Boudin Blanc Terrine with Red Onion Confit,"[75.962036, 87.23587, -14.523556, 125.86491, 9...","['1 1/2 cups whipping cream', '2 medium onions...","[whipping, cream, onions, bay, leaves, cloves,...",['Combine first 9 ingredients in heavy medium ...,84
2,Potato and Fennel Soup Hodge,"[23.013504, 58.24837, -29.56066, 73.09149, 33....","['1 fennel bulb (sometimes called anise), stal...","[fennel, bulb, sometimes, called, anise, stalk...",['In a large heavy saucepan cook diced fennel ...,19
3,Mahi-Mahi in Tomato Olive Sauce,"[21.615114, 3.2711647, -58.65011, 101.31643, 7...","['2 tablespoons extra-virgin olive oil', '1 cu...","[extra-virgin, olive, oil, onion, wine, anchov...",['Heat oil in heavy large skillet over medium-...,96
4,Spinach Noodle Casserole,"[28.740934, 3.7594368, 12.923091, 16.016447, 1...","['1 12-ounce package frozen spinach soufflé, t...","[12-ounce, package, frozen, spinach, soufflé, ...",['Preheat oven to 350°F. Lightly grease 8x8x2-...,0
5,The Best Blts,"[42.969856, -7.9991136, -16.851566, 81.6432, -...",['2 1/2 cups (lightly packed) fresh basil leav...,"[lightly, basil, leaves, mayonnaise, butter, '...","['Mix basil, mayonnaise and butter in processo...",92
6,Ham and Spring Vegetable Salad with Shallot Vi...,"[2.262138, 54.973976, 14.464672, 82.10351, -10...","['1 1/2 pounds small red-skinned potatoes, eac...","[red-skinned, potatoes, wedges, baby, carrots,...",['Cook potatoes and carrots in large pot of bo...,43
7,Spicy-Sweet Kumquats,"[36.92724, -8.703801, -16.14729, 10.904221, 15...","['6 tablespoons granulated sugar', '1 1/2 tabl...","[granulated, sugar, chili, powder, powdered, e...",['Stir together sugar and chili powder. Whisk ...,40
8,Korean Marinated Beef,"[-6.5129356, -26.902327, -47.21583, 80.02176, ...","['1/4 cup soy sauce', '1 tablespoon sugar', '2...","[soy, sauce, sugar, asian, sesame, oil, scalli...","['Stir together soy sauce, sugar, sesame oil, ...",66
9,Ham Persillade with Mustard Potato Salad and M...,"[79.41853, 34.413067, 13.321831, 91.96728, 19....","['6 long parsley sprigs, divided', '1 3/4 cups...","[long, parsley, sprigs, reduced-sodium, chicke...",['Chop enough parsley leaves to measure 1 tabl...,43


In [12]:
x=widgets.Text(
    value='hungry for ?',
    placeholder='Type something',
    description='First Ingredient:',
    disabled=False
)

y=widgets.Text(
    value='hungry for ?',
    placeholder='Type something',
    description='Second Ingredient:',
    disabled=False
)

In [123]:
recipe=recipe[recipe['clustered_groups']==num]

In [124]:
recipe

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
65,Orange-Almond Cake with Chocolate Icing,"[141.10497, 52.806316, -2.1063466, 38.491425, ...","['3 large oranges', '2 cups plus 2 tablespoons...","[oranges, purpose, flour, almonds, powder, gen...",['Preheat oven to 350°F. Butter three 9-inch-d...,15
75,Banana Layer Cake with Cream Cheese Frosting,"[115.98383, 15.810266, 4.4264674, 43.45575, -5...","['2 1/4 cups cake flour', '3/4 teaspoon baking...","[cake, flour, soda, powder, pinch, mashed, rip...",['Preheat oven to 350°F. Lightly butter three ...,15
334,Ginger Crunch Cake with Strawberry Sauce,"[139.52003, -5.6976523, 57.73746, 51.35937, -6...","['1 cup pecans, toasted and chopped fine', '1/...","[pecans, toasted, fine, gingersnap, cookie, cr...",['Preheat oven to 350°F. Grease three 8- by 2-...,15
378,Classic Coconut Cake,"[147.13603, 22.55267, 29.90657, 37.63743, -20....","['Nonstick vegetable oil spray', '2 cups all p...","['nonstick, vegetable, oil, spray, purpose, fl...",['Preheat oven to 350°F. Coat two 9-inch-diame...,15
419,Lemon Cornmeal Cake with Lemon Glaze and Crush...,"[90.37719, 25.653423, 29.99604, 68.20867, 83.7...","['1 1/2 cups (packed) powdered sugar, sifted',...","[powdered, sugar, sifted, lemon, juice, purpos...",['Combine powdered sugar and 2 tablespoons lem...,15
420,Amaretto Olive Oil Cake,"[46.182774, 32.887966, 41.22029, 37.02787, 7.9...",['1 cup (4 1/2 ounces) blanched slivered almon...,"[blanched, slivered, almonds, matzoh, cake, me...",['Heat oven to 325°F with rack in middle. Grea...,15
515,Sour Cream Layer Cake with Pecan Brittle,"[150.11766, 40.098167, 29.933968, 32.833405, 3...","['Nonstick vegetable oil spray', '3/4 cup suga...","['nonstick, vegetable, oil, spray, sugar, wate...",['Spray baking sheet with nonstick spray. Comb...,15
1534,Orange-Spice Bûche de Noël with Orange Cream C...,"[94.06328, 55.763203, 74.340515, 114.93197, 71...","['2 oranges', '2/3 cup water', '1/2 cup sugar'...","[oranges, water, sugar, apple, cider, vinegar,...","['Using vegetable peeler, remove rind from ora...",15
2014,Blueberry-Lemon Cake with Lemon Cream Cheese F...,"[110.24114, 54.91444, 37.1743, 50.0735, 37.515...","['2 cups plus 6 tablespoons cake flour', '2 te...","[cake, flour, powder, blueberries, three, 4.4-...",['Preheat oven to 350°F. Butter three 9-inch-d...,15
2225,Carrot Cake with Cream Cheese Frosting,"[120.25094, 53.967266, 83.15286, 119.52622, 29...","['1 tablespoon unsalted butter, for greasing t...","[butter, greasing, pans, unbleached, flour, po...",['Position a rack in the middle of the oven an...,15


In [15]:
def word_clouder(num=0):
    group=recipe[recipe['clustered_groups']==num]['title']
    processed_words=group.apply(lambda x: process_article(x))
    articles_concat = []
    for article in processed_words:
        articles_concat += article
    word_list=Counter(articles_concat).most_common(100)
    word_list_2=dict(word_list)
    wordcloud = WordCloud().generate_from_frequencies(word_list_2)
    plt.figure(figsize=[20,10])
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    return plt.show()


In [16]:
def recipe_filter(recipe,x,y=x):
    x=x.value
    y=y.value
    test_run=recipe[recipe['ingredients'].str.contains(x)]
    test_run=test_run[test_run['ingredients'].str.contains(y)]
    test_run=test_run.reset_index(drop=True)
    return test_run

In [17]:
def recipe_generator(data):


    item=randint(0,(len(data)-1))

    test_ingredients=data['processed_ingredients'][item]

    key=test_ingredients[0]

    answer=model_combo.wv.most_similar(positive=[test_ingredients[0]])

    final=answer[0][0]

    similar_words=model_combo.wv.most_similar(positive=[key])

    similar_list=[]
    words=[]
    for word in range(1,len(similar_words)):
        similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
        words.append(similar_words[word][0])

    layered_relationships=dict(zip(words,similar_list))

    final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))

    data.iloc[item]['title'],data.iloc[item]['ingredients']

    return data.iloc[item]['title'].replace(key,final_2), data.iloc[item]['directions'].replace(key,final_2), data.iloc[item]['ingredients'].replace(key,final_2)

In [18]:
#p=interact(word_clouder,num=(0,100,1))

In [116]:
w = widgets.IntSlider()
interact(word_clouder,num=w)


interactive(children=(IntSlider(value=0, description='num'), Output()), _dom_classes=('widget-interact',))

<function __main__.word_clouder(num=0)>

In [131]:
play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])
interact(word_clouder,num=play)

interactive(children=(Play(value=50, description='Press play'), Output()), _dom_classes=('widget-interact',))

<function __main__.word_clouder(num=0)>

In [118]:
num=w.value
recipe[]

In [119]:
num

15

In [21]:
x

Text(value='hungry for ?', description='First Ingredient:', placeholder='Type something')

In [22]:

y



Text(value='hungry for ?', description='Second Ingredient:', placeholder='Type something')

In [105]:
y.value

''

In [106]:
button

Button(description='Create next input', style=ButtonStyle())

In [120]:
fred=recipe_filter(x,y)

In [121]:
fred

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
0,"Lentil, Apple, and Turkey Wrap","[10.785597, 21.088675, -40.898643, 66.33199, 1...",['4 cups low-sodium vegetable or chicken stock...,"[low-sodium, vegetable, chicken, stock, dried,...","['1. Place the stock, lentils, celery, carrot,...",23
1,Boudin Blanc Terrine with Red Onion Confit,"[75.962036, 87.23587, -14.523556, 125.86491, 9...","['1 1/2 cups whipping cream', '2 medium onions...","[whipping, cream, onions, bay, leaves, cloves,...",['Combine first 9 ingredients in heavy medium ...,84
2,Potato and Fennel Soup Hodge,"[23.013504, 58.24837, -29.56066, 73.09149, 33....","['1 fennel bulb (sometimes called anise), stal...","[fennel, bulb, sometimes, called, anise, stalk...",['In a large heavy saucepan cook diced fennel ...,19
3,Mahi-Mahi in Tomato Olive Sauce,"[21.615114, 3.2711647, -58.65011, 101.31643, 7...","['2 tablespoons extra-virgin olive oil', '1 cu...","[extra-virgin, olive, oil, onion, wine, anchov...",['Heat oil in heavy large skillet over medium-...,96
4,Spinach Noodle Casserole,"[28.740934, 3.7594368, 12.923091, 16.016447, 1...","['1 12-ounce package frozen spinach soufflé, t...","[12-ounce, package, frozen, spinach, soufflé, ...",['Preheat oven to 350°F. Lightly grease 8x8x2-...,0
5,The Best Blts,"[42.969856, -7.9991136, -16.851566, 81.6432, -...",['2 1/2 cups (lightly packed) fresh basil leav...,"[lightly, basil, leaves, mayonnaise, butter, '...","['Mix basil, mayonnaise and butter in processo...",92
6,Ham and Spring Vegetable Salad with Shallot Vi...,"[2.262138, 54.973976, 14.464672, 82.10351, -10...","['1 1/2 pounds small red-skinned potatoes, eac...","[red-skinned, potatoes, wedges, baby, carrots,...",['Cook potatoes and carrots in large pot of bo...,43
7,Spicy-Sweet Kumquats,"[36.92724, -8.703801, -16.14729, 10.904221, 15...","['6 tablespoons granulated sugar', '1 1/2 tabl...","[granulated, sugar, chili, powder, powdered, e...",['Stir together sugar and chili powder. Whisk ...,40
8,Korean Marinated Beef,"[-6.5129356, -26.902327, -47.21583, 80.02176, ...","['1/4 cup soy sauce', '1 tablespoon sugar', '2...","[soy, sauce, sugar, asian, sesame, oil, scalli...","['Stir together soy sauce, sugar, sesame oil, ...",66
9,Ham Persillade with Mustard Potato Salad and M...,"[79.41853, 34.413067, 13.321831, 91.96728, 19....","['6 long parsley sprigs, divided', '1 3/4 cups...","[long, parsley, sprigs, reduced-sodium, chicke...",['Chop enough parsley leaves to measure 1 tabl...,43


In [104]:
recipe_generator(fred)

('Grilled Vegetable Salad with Greens, Tomatoes, Herbs, Olives and Cheese ',
 "['Prepare barbeque (medium-high heat). Whisk 3/4 cup oil and vinegar in medium bowl to blend. Place onion, beets, zucchini, eggplants, and red bell pepper on baking sheet. Brush both sides with some of vinaigrette. Sprinkle vegetables with salt and pepper. Grill vegetables until just cooked through, about 10 minutes per side for beets, 6 minutes per side for onion and 4 minutes per side for zucchini, eggplant, and peppers. (Vegetables can be grilled 1 hour ahead. Let stand at room temperature.) Brush bread with additional 'vegetable oil; sprinkle with pepper. Grill bread until beginning to brown, about 2 minutes per side.', 'Arrange greens on large platter. Overlap tomatoes atop greens in center of platter. Sprinkle tomatoes with salt and pepper. Arrange grilled vegetables atop greens around edge of platter. Drizzle remaining vinaigrette over tomatoes and grilled vegetables. Sprinkle tomatoes and vegetables 

In [82]:
recipe[recipe['title']=='Grilled Open-Faced Vegetable, Pesto, and Mozzarella Sandwiches ']['ingredients']

18142    ['1 red bell pepper, quartered', '1 yellow bel...
Name: ingredients, dtype: object

In [83]:
recipe.iloc[18142]['ingredients']

"['1 red bell pepper, quartered', '1 yellow bell pepper, quartered', '3 small eggplants (about 1/2-pound total), sliced diagonally 1/4inch thick', 'olive oil for brushing vegetables and bread', '1 14-inch-long loaf Italian bread, halved horizontally with serrated knife', '1/2 cup pesto', '1 cup diced mozzarella cheese (about 1/4 pound)', '3 tablespoons finely shredded fresh basil leaves for garnish', '1 cup packed fresh basil leaves', '3 tablespoons pine nuts or chopped walnuts, toasted lightly and cooled', '1 large garlic clove, chopped', '3 tablespoons freshly grated Parmesan cheese', '1/3 olive oil']"

In [84]:
widgets.Button(
    description='Click me',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

Button(button_style='success', description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [85]:
widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [86]:
click

Button(description='Create next input', style=ButtonStyle())

In [88]:
from IPython.core.display import HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Refresh"></form>''')

In [89]:
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>